### Задание

1.Взять датасет

https://huggingface.co/datasets/merionum/ru_paraphraser 

решить задачу парафраза

In [1]:
!pip install transformers sentencepiece datasets -q


     |████████████████████████████████| 5.8 MB 4.2 MB/s 
     |████████████████████████████████| 1.3 MB 22.8 MB/s 
     |████████████████████████████████| 451 kB 37.9 MB/s 
     |████████████████████████████████| 182 kB 68.4 MB/s 
     |████████████████████████████████| 7.6 MB 35.8 MB/s 
     |████████████████████████████████| 212 kB 59.6 MB/s 
     |████████████████████████████████| 132 kB 52.7 MB/s 
     |████████████████████████████████| 127 kB 51.3 MB/s 


In [2]:
import pandas as pd
from datasets import load_dataset

import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer

In [3]:
corpus = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-62b622e3b2274f4e/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

In [5]:
result_df = []

def paraphrase_check(row_numb):
    
    text1 = corpus['train']['text_1'][row_numb]
    text2 = corpus['train']['text_2'][row_numb]
    
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).numpy()
    
    p = proba[0]
    
    result_df.append([text1, text2, p[1]])
    
    return pd.DataFrame(result_df, columns=['текст 1', 'текст 2', 'вероятность парафраза'])

In [6]:
paraphrase_check(0)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301


In [7]:
paraphrase_check(10)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294


In [8]:
paraphrase_check(20)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294
2,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144


In [9]:
paraphrase_check(30)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294
2,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144
3,"Прокуратура пока не решила, обжаловать ли приг...",Защита Васильевой обжалует приговор суда,0.239415


In [10]:
paraphrase_check(40)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294
2,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144
3,"Прокуратура пока не решила, обжаловать ли приг...",Защита Васильевой обжалует приговор суда,0.239415
4,Владимир Путин освободил от должности почти 20...,Путин снял с должностей более 20 руководителей...,0.972911


Предобученная модель хорошо решает задачу